## Mesh

In [1]:
from ngsolve import *
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.internal import *
from xfem import *
import numpy as np
import ipywidgets
from ngsolve.solvers import *
import gc
def MakeGeometry():
    geo = SplineGeometry()

    # Define Points:|
    p0 = geo.AppendPoint(0.0,     0.0)
    p0_inset = geo.AppendPoint(0.00005,0.0)
    p1 = geo.AppendPoint(0.001,   0.0)
    p1b = geo.AppendPoint(0.002,  0.0)
    p2 = geo.AppendPoint(0.007, 0.0)
    p3_inset = geo.AppendPoint(0.00995,0.0)
    p3 = geo.AppendPoint(0.01, 0.0)
    
    p4 = geo.AppendPoint(0.01, 0.10)
    p5 = geo.AppendPoint(0.008, 0.10)
    p1b_top = geo.AppendPoint(0.002,  0.10)
    
    p6 = geo.AppendPoint(0.001,   0.10)
    p7 = geo.AppendPoint(0.00,   0.10)
    p7_inset = geo.AppendPoint(0.00005,0.10)

    # Bottom edges
 
    geo.Append(["line", p0, p1],     leftdomain=1, rightdomain=0, bc="Liq_Outlet_a")
    geo.Append(["line", p1, p1b],    leftdomain=2, rightdomain=0, bc="Liq_Outlet_b")  # new domain
    geo.Append(["line", p1b, p2],    leftdomain=3, rightdomain=0, bc="bottom")
    geo.Append(["line", p2, p3_inset],     leftdomain=4, rightdomain=0, bc="Gas_Inlet")
    geo.Append(["line",p3_inset,p3], leftdomain=4 ,rightdomain=0, bc = "Gas_Buffer")
    
    # Right edge
    geo.Append(["line", p3, p4],     leftdomain=4, rightdomain=0, bc="right")

    # Top edges
    geo.Append(["line", p4, p5],     leftdomain=4, rightdomain=0, bc="Gas_Outlet")
    geo.Append(["line", p5, p1b_top],leftdomain=3, rightdomain=0, bc="top")
    geo.Append(["line", p1b_top, p6],leftdomain=2, rightdomain=0, bc="top")     # new domain
    geo.Append(["line", p6, p7_inset],     leftdomain=1, rightdomain=0, bc="Liq_Inlet")
    geo.Append(["line", p7_inset,p7],     leftdomain=1, rightdomain=0, bc="Liq_Buffer")

    # Left edge
    geo.Append(["line", p7, p0],     leftdomain=1, rightdomain=0, bc="left")

    # Internal vertical separators
    interface = geo.Append(["line", p1, p6],       leftdomain=1, rightdomain=2)  # 
    geo.Append(["line", p1b, p1b_top], leftdomain=2, rightdomain=3)  # 
    geo.Append(["line", p2, p5],       leftdomain=3, rightdomain=4)  #



    # Set mesh size per domain

    geo.SetDomainMaxH(1, 0.00009)
    geo.SetDomainMaxH(2, 0.00009)  # new domain (optional: adjust mesh size)
    geo.SetDomainMaxH(3, 0.0007)
    geo.SetDomainMaxH(4, 0.0005)

    return geo.GenerateMesh()


mesh =  Mesh(MakeGeometry())

importing ngsxfem-2.1.2405


## Setup

In [2]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [3]:
# =============================================================================
# FINITE ELEMENT SPACES
# =============================================================================

# Navier-Stokes spaces
V = VectorH1(mesh, order=2, dirichlet="Liq_Inlet|Gas_Inlet|top|bottom|left|right|Liq_Buffer|Gas_Buffer") #dirichlety = "top|bottom"
Q = H1(mesh, order=1,dirichlet="")  # pressure constraints
X = V*Q  # Mixed velocity-pressure space

# AllenCahn Space
L = H1(mesh, order=2,dirichlet="left|Liq_Inlet|Gas_Inlet|Gas_Outlet|Gas_Buffer|top") 

# GRID FUNCTIONS
# =============================================================================

# Navier-Stokes solutions
u_p = GridFunction(X)      # Current [u, p] solution
u_p_old = GridFunction(X)  # Previous time step [u, p]

# Phase field solutions  
phi = GridFunction(L)      # Current [phi, mu] solution
phi_old = GridFunction(L)  # Previous time step [phi, mu]
# Boundary condition helpers|
u_D = GridFunction(V)  # Velocity BC helper
p_D = GridFunction(Q)  # Pressure BC helper


In [4]:
# =============================================================================
# INITIALIZATION
# =============================================================================
epsilon = 0.00006
d = x - 0.001         # signed distance
s = d / (2 * epsilon)      # scaled distance
phi_init = (1 - exp(-2*s)) / (1 + exp(-2*s))

phi.Set(phi_init)
phi_old.Set(phi_init)

# Initialize Navier-Stokes (can be zero or small perturbation)
u_p.components[0].Set(CoefficientFunction((0.0, 0.0)))  # Initial velocity
u_p.components[1].Set(0.0)             # Initial pressure
u_p_old.vec.data = u_p.vec

In [5]:
X.ndof

294020

In [6]:
mu_l = 0.001 #Pascal*seconds (liquid viscosity)
mu_g = 1.8e-5 #Pa*s (gas viscosity)
rho_l = 997.0 #Kg/m3 (liquid density)
rho_g = 1.2 #Kg/m3 (gas density)
surface_tension = 0.072 #Surface tension force N/m

## Solver

In [7]:
def BuildNavierStokes(X, dt,
                      v_test, q_test,
                      u_trial, p_trial,
                      u_p_old,
                      rho, mu,g):        #

    """
    Builds the weak-form Navier–Stokes residual suitable for Newton,
    including a weak velocity BC on `inlet_tag` if `gas_inlet_cf` is given.
    """

    u_old = u_p_old.components[0]

    h = specialcf.mesh_size                 # mesh size function

    a_ns = BilinearForm(X)

    # Time derivative
    a_ns += (rho / dt) * (u_trial - u_old) * v_test * dx

    # Convection
    a_ns += rho * InnerProduct(grad(u_trial) * u_trial, v_test) * dx

    # Viscous term
    a_ns += mu * InnerProduct(Sym(grad(u_trial)), Sym(grad(v_test))) * dx

    # Pressure terms
    a_ns += (-p_trial * div(v_test)) * dx
    a_ns += ( div(u_trial) * q_test ) * dx

    # Gravity
    a_ns += -(rho * g * v_test) * dx

    return a_ns


In [8]:
Draw(phi,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [9]:
def UpdateMaterialParameters(phi, rho_l, rho_g, dx):
    """Get smoothed material properties using Heaviside function"""
    eps = 3 * dx
    H = IfPos(phi + eps, 
              IfPos(eps - phi,
                    0.5 * (1 + phi/eps + sin(pi * phi/eps) / pi),
                    1.0),
              0.0)
    return rho_l + (rho_g - rho_l) * H

In [10]:
# -------------------------------------------------------------
ramp_steps  = 30               # number of ramp-up steps
ramp_type   = "tanh"           # or "linear", "tanh"
# -------------------------------------------------------------

def ramp_scale(step,ramp_steps,ramp_type):
    if step >= ramp_steps:
        return 1.0
    if ramp_type == "linear":
        return step / ramp_steps
    if ramp_type == "halfcos":
        from math import cos, pi
        return 0.5*(1 - cos(pi * step / ramp_steps))
    if ramp_type == "tanh":
        from math import tanh
        delta = 0.15 * ramp_steps    # smoothness width in step units
        return 0.5 * (1 + tanh((step - 0.5 * ramp_steps) / delta))
    raise ValueError("Unknown ramp type")


In [11]:
import time
from ngsolve import TaskManager, SetNumThreads
SetNumThreads(18)
import multiprocessing
print("Logical CPUs (threads):", multiprocessing.cpu_count())


Logical CPUs (threads): 16


In [12]:
def BuildAllenCahn(AC_space, dt, phi_old, vel_cf, tau, eps,
                   inflow_penalty=1.0, inlet_group="Liq_Inlet",phi_inlet_cf = phi_init):
    phi, w = AC_space.TnT()
    a_ac = BilinearForm(AC_space)

    # Base terms
    a_ac += ((phi - phi_old) / dt) * w * dx
    a_ac += InnerProduct(vel_cf, grad(phi)) * w * dx
    a_ac += (1.0 / tau) * (phi - phi**3) * w * dx
    a_ac += (eps**2 / tau) * InnerProduct(grad(phi), grad(w)) * dx

    # SUPG (volume only)
    hK   = specialcf.mesh_size
    umag = Norm(vel_cf)
    tau_supg = 1.0 / sqrt( (2.0/dt)**2 + (2.0*umag/hK)**2 )
    R_no_diff = ((phi - phi_old)/dt
                 + InnerProduct(vel_cf, grad(phi))
                 + (1.0/tau)*(phi - phi**3))
    a_ac += tau_supg * InnerProduct(vel_cf, grad(w)) * R_no_diff * dx

    # Inflow upwind penalty (your phi_inlet_cf is assumed defined)
    mesh = AC_space.mesh
    n  = specialcf.normal(mesh.Boundaries(inlet_group))
    un = InnerProduct(vel_cf, n)           # >0 outflow, <0 inflow
    inflow = IfPos(-un, -un, 0)            # max(0, -u·n)

    a_ac += inflow_penalty * inflow * (phi - phi_inlet_cf) * w * ds(inlet_group)


    return a_ac



In [13]:
    #Trial n Test

test_functions = X.TestFunction()

trial_functions = X.TrialFunction()

v_test,q_test = test_functions # for velocity, and pressure

u_trial,p_trial = trial_functions #for velocity, and pressure

def cosine_wave(x, f):
    phase_shift = 1 / (2 * f)  # shifts the cosine so it starts at zero
    return 0.5 * (np.cos(2 * np.pi * f * (x - phase_shift)) + 1)

def ClampPhi(phi_gf, lo=-1.0, hi=1.0):
    """Clamp phase field values in-place to [lo, hi]."""
    arr = phi_gf.vec.FV().NumPy()
    arr[:] = np.minimum(np.maximum(arr, lo), hi)



In [14]:
Draw(grad(phi),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [ ]:

dt = CoefficientFunction(1e-4)
dt_value = 1e-4

h = specialcf.mesh_size
t_curr= 0

Umax_target = 0.42
step, steps = 0, 2000
spinup = 300
start_time = time.time()
freq = 50 #hz


#arr = np.load("Navier_Stokes_GFU.npy")   
#u_p_old.vec.FV().NumPy()[:] = arr           # ← in-place copy
#u_p.vec.FV().NumPy()[:] = arr           # ← in-place copy

#phi_arr = np.load("Phi_Thin.npy")   
#phi.vec.FV().NumPy()[:] = phi_arr           # ← in-place copy


# Your main loop with VOF update added:
u_p.components[1].vec[:] = 0

with TaskManager():
    while step < steps:
        # ------------------------------------------------------------------
        # 1. UPDATE MATERIAL PROPERTIES
        # ------------------------------------------------------------------

    
        t_curr += dt_value

        S = ramp_scale(step,50,"tanh")
        Sg = ramp_scale(step,50,"tanh")
        
        g = CoefficientFunction((0.0, Sg*-9.807))   # gravitational acceleration
        
        #jump = cosine_wave(t_curr,freq) * Umax_target + 1e-6

        u_max = S * Umax_target #+ jump
        
        rho_smoothed = UpdateMaterialParameters(phi, rho_l, rho_g,0.3)
        mu_smoothed = UpdateMaterialParameters(phi, mu_l, mu_g,0.3)

        # ------------------------------------------------------------------
        # 2. NAVIER–STOKES SOLVE (unchanged)
        # -----------------------------------------------------------------


        #COMPUTE PARABOLA FOR LIQ IN
        xmin = 0.0 + 5e-5
        xmax = 0.001
        
        xhat = (x - xmin) / (xmax - xmin)

        parabola = IfPos(x - xmin,
                 IfPos(xmax - x, 4 * xhat * (1 - xhat), 0),
                 0)
        
        #COMPUTE PARABOLA FOR GAS IN
        
        xmin_g = 0.007
        xmax_g = 0.010 - 5e-5
        
        xhat_g = (x - xmin_g) / (xmax_g - xmin_g)
        
        parabola_g = IfPos(x - xmin_g,
                 IfPos(xmax_g - x, 4 * xhat_g * (1 - xhat_g), 0),
                 0)
        
        uin = CoefficientFunction((0, -1*u_max*parabola))
        uin_g = CoefficientFunction((0, 0.9*Sg*parabola_g))

        u_bc = CF((
            IfPos(0.002 - x, uin[0], uin_g[0]),  # x-component (both 0 in your case)
            IfPos(0.002 - x, uin[1], uin_g[1])   # y-component
        ))
                
        
        A_ns = BuildNavierStokes(X, dt, v_test, q_test, u_trial, p_trial,
                                u_p_old, rho_smoothed, mu_smoothed,g)
        
        #print(f"[Step {step}]")

        u_p.components[0].Set(u_bc, definedon=mesh.Boundaries("Liq_Inlet|Gas_Inlet"))
        damp = 0.5
        
        Newton(A_ns, u_p,
               freedofs=X.FreeDofs(),
               maxit=12, maxerr=1e-8,
               inverse="pardiso", dampfactor=damp,
               printing=True if step % 50 == 0 else False )

        
        u = u_p.components[0]

 
        vel_cf = u                                # use solved velocity field

        if step > spinup:
            

            A_ac = BuildAllenCahn(AC_space  = L,
                                  dt        = dt_value,
                                  vel_cf    = vel_cf,
                                  tau       = 1,
                                  phi_old   = phi_old,
                                  eps       = epsilon,)            
            Newton(A_ac, phi,
                   freedofs   = L.FreeDofs(),
                   maxit      = 12,
                   maxerr     = 1e-8,
                   inverse    = "pardiso",
                   dampfactor = 0.1,
                   printing   = False)


            ClampPhi(phi, lo=-1.0, hi=1.0)
            # roll phase field forward
            phi_old.vec.data = phi.vec.data
            
        if  step % 500 == 0 :
            print(f'LEVEL SET {step}')
            #DrawDC(phi_old,-1,1,mesh)
            #Draw(u_p.components[0][1],mesh)
           # Draw(u_p.components[1],mesh)
            #DrawDC(phi_old,-1,1,mesh)
            #Draw(phi_old,mesh)
            print(f"Time to get here {start_time - time.time()}")

            
        u_p_old.vec.data = u_p.vec.data
        
    
        step += 1
        #print(f"Starting  Step {step+1}")

Newton iteration  0
err =  0.000890841730956217
Newton iteration  1
err =  0.00044542394567896316
Newton iteration  2
err =  3.862986343156098e-08
Newton iteration  3
err =  2.1930432159262338e-14
LEVEL SET 0
Time to get here -7.536816596984863


In [ ]:
Draw(phi,mesh)

In [ ]:
Draw(u_p_old.components[0],mesh)

In [ ]:
np.save("Phi_Thin.npy", phi_old.vec.FV().NumPy())
np.save("Matching_NS.npy",u_p_old.vec.FV().NumPy())